### 채용공고, 자격요건및우대사항 요약

#### 직업별 DF

In [34]:
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk

In [3]:
gongo = pd.read_csv('..\\preprocessing\\preprocessed_gongo_final.csv')
job = pd.read_csv('..\\preprocessing\\job_lv2_df.csv')

In [6]:
# job list
job_lv2_list = job.columns
job_lv2_list

Index(['소프트웨어 개발', '마케팅', '백엔드 개발', '프론트엔드 개발', '서비스 기획', 'PM/PO',
       '상품개발/기획/MD', '국내영업', '웹개발', '네트워크/보안/운영',
       ...
       '인쇄/출판/편집 ', '자산운용가', '비서', '작가/시나리오 ', '사무보조/문서작성 ', '제과제빵사', '제품별마케팅',
       '보상', '치기공사/치과위생사', '금융/보험영업'],
      dtype='object', length=146)

In [7]:
# make a foler if it's not existed
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [17]:
# make a folder
folderpath = './gongo_by_job'
makedirs(folderpath)

# make a csv file by same job
for job_name in job_lv2_list:
    
    # init
    df = pd.DataFrame()

    # job index list
    idx = job[job[job_name] == 1].index
    
    for i in idx:
        df = pd.concat([df, gongo[gongo['index']==i]])
    
    # save as a csv file
    df.to_csv('{}/{}.csv'.format(folderpath, job_name.replace('/','').replace(' ','')), index=False)

#### 직업별 채용공고, 자격및우대사항 요약

In [42]:
# make a folder
folderpath = './job_work_qual'
makedirs(folderpath)

path = './gongo_by_job/'
file_list = os.listdir(path)
file_list_name = [file for file in file_list if file.endswith('.csv')]

# by jobs
for filename in file_list_name:
	df = pd.read_csv('{}{}'.format(path, filename))
	work_sen_all = '\n'.join(df['주요 업무'])
	qual_sen_all = '\n'.join(df['자격요건 및 우대사항'])

	# model
	model_dir = "lcw99/t5-large-korean-text-summary"
	tokenizer = AutoTokenizer.from_pretrained(model_dir)
	model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

	# parameters
	max_input_length = 512

	# summarize work sentences
	inputs_w = ["summarize_summary: " + work_sen_all]  # summarize
	inputs_w = tokenizer(inputs_w, max_length=max_input_length, truncation=True, return_tensors="pt")
	output_w = model.generate(**inputs_w, num_beams=8, do_sample=True, min_length=10, max_length=300)
	decoded_output_w = tokenizer.batch_decode(output_w, skip_special_tokens=True)[0]
	predicted_w = nltk.sent_tokenize(decoded_output_w.strip())[0]

	# summarize qualification sentences
	inputs_q = ["summarize_summary: " + qual_sen_all]  # summarize
	inputs_q = tokenizer(inputs_q, max_length=max_input_length, truncation=True, return_tensors="pt")
	output_q = model.generate(**inputs_q, num_beams=8, do_sample=True, min_length=10, max_length=300)
	decoded_output_q = tokenizer.batch_decode(output_q, skip_special_tokens=True)[0]
	predicted_q = nltk.sent_tokenize(decoded_output_q.strip())[0]

	# save a predicted sentence as a text file
	with open('{}/{}_work.txt'.format(folderpath,filename.replace('.csv','')),'w') as f:
		f.write(predicted_w)
	f.close()

	with open('{}/{}_qual.txt'.format(folderpath,filename.replace('.csv','')),'w') as f:
		f.write(predicted_q)
	f.close()